# Load Data with Vectors

## Get keys and urls

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}")
print(f"OpenAI URL: {OPENAI_URL}")

Weaviate Key:root-user-key
OpenAI API Key: sk-proj-iuwKF1Q94jnW
OpenAI URL: https://api.openai.com


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

WeaviateStartUpError: Could not connect to Weaviate:Connection to Weaviate failed. Details: .

In [ ]:
from weaviate.classes.config import Configure

def create_wiki_collection():
    if client.collections.exists("Wiki"):
        client.collections.delete("Wiki")

    # Create a collection here - with OpenAI vectorizer and define source properties
    client.collections.create(
        name="Wiki",

        vector_config=[
            Configure.Vectors.text2vec_openai(
                name="main_vector",
                base_url=OPENAI_URL,
                model="text-embedding-3-small",
                source_properties=['title', 'text'] # which properties should be used to generate a vector
            )
        ],
    )

create_wiki_collection()

## Load the data from parquet files

In [ ]:
from datasets import load_dataset

def prepare_dataset():
    return load_dataset('parquet', data_files={'train': ['../../wiki-data/openai/text-embedding-3-small/*.parquet']}, split="train", streaming=True)
    # return load_dataset("weaviate/wiki-sample", "openai-text-embedding-3-small", split="train", streaming=True)

### Dataset Test
<!-- The parquet files should be located in "datasets/openai". -->

In [ ]:
dataset = prepare_dataset()

counter = 10
for i in dataset:
    print(i)

    counter -= 1
    if(counter == 0): break

### The import function

`TODO:`
* add a function to add objects to batch

In [ ]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

def import_wiki_data(max_rows=20_000):
    print(f"Importing {max_rows} data items")

    dataset = prepare_dataset()
    wiki = client.collections.get("Wiki")

    counter = 0

    with wiki.batch.fixed_size(batch_size=2000, concurrent_requests=2) as batch:
        for item in tqdm(dataset, total=max_rows):

            data_to_insert = {   
                "wiki_id": item["wiki_id"],
                "text": item["text"],
                "title": item["title"],
                "url": item["url"],
            }

            item_id = generate_uuid5(item["wiki_id"])

            # vector = item["vector"]
            item_vector = {
                "main_vector": item["vector"]
            }

            batch.add_object(
                properties=data_to_insert,
                
                uuid=item_id,
                vector=item_vector
            )

            # Check number of errors while running
            if(batch.number_errors > 10):
                print(f"Reached {batch.number_errors} Errors during batch import")
                break
            
            # stop after the request number reaches = max_rows
            counter += 1
            if(counter >= max_rows):
                break
    
    # check for errors at the end
    if (len(wiki.batch.failed_objects)>0):
        print("Final error check")
        print(f"Some errors {len(wiki.batch.failed_objects)}")
        print(wiki.batch.failed_objects[-1])
    
    print(f"Imported {counter} items")
    print("-----------------------------------")

In [ ]:
import_wiki_data(20_000)

## Check if data loaded correctly

In [ ]:
wiki = client.collections.get("Wiki")
len(wiki)

In [ ]:
res = wiki.query.fetch_objects(limit=1, include_vector=True)
print(res.objects[0].properties)
print(res.objects[0].vector)

## Close the client

In [ ]:
client.close()